<a href="https://colab.research.google.com/github/RiaStevens/rmt-research/blob/main/RMT_Sim_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from google.colab import files, drive
import numpy.linalg as npla
from scipy.optimize import curve_fit
import json
from ipywidgets import widgets, interactive
from IPython.utils import io

data_dir = "/content/drive/MyDrive/NNH Project/Full Data/"

# Setting up Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd $data_dir

/content/drive/MyDrive/NNH Project/Full Data


In [ ]:
# in case directory is lost
"""
import os
path = "/content" # /content is pretty much the root
os.chdir(path)
"""

'\nimport os\npath = "/content" # /content is pretty much the root\nos.chdir(path)\n'

# Key Function Definitions

## Simulation Definitions (unnecessary to run)

In [ ]:
def gradient_descent(W1, W2, x, y, num_iters, step, prop, e, id):
  (n2, n1) = W2.shape
  (n0, m) = x.shape

  losses = np.empty(0)
  for j in range(num_iters):
    if j in [0, 10, 100, 1000, 2500, 5000, 10000]:
      %mkdir 'iters_{j}'
      %cd 'iters_{j}'
      string = f'id_{id}'
      save_to_file(W1, W2, string)
      %cd '..'

    G1, G2 = gradient(x, W2, e, prop)
    W1 = W1 - G1 * step
    W2 = W2 - G2 * step
    
    z = W1 @ x
    prop = np.maximum(z, np.zeros_like(z))
    y_hat = W2 @ prop
    e = y_hat - y

    loss = npla.norm(e) ** 2 / (2 * m)
    losses = np.append(losses, loss)
    
  return W1, W2, losses, e, prop

## Analysis Definitions

In [ ]:
def hessian(W1, W2, x, y, l, l0):
  (n2, n1) = W2.shape
  (n0, m) = x.shape

  z = W1 @ x
  prop = np.maximum(z, np.zeros_like(z))
  y_hat = W2 @ prop
  e = y_hat - y 

  def compute_H0():
    J_1 = np.tensordot(W2, x, axes=0)
    for a, u in itertools.product(range(n1), range(m)):
      if prop[a, u] == 0:
        J_1[0:n2, a, 0:n0, u] = 0

    J_2 = np.tensordot(np.ones((n2, n2)), prop, axes=0)

    H_0_tl = np.einsum('xaby,xcdy->abcd', J_1, J_1)
    H_0_tl_flat = H_0_tl.reshape((n1 * n0, n1 * n0))

    H_0_tr = np.einsum('xaby,xcdy->abcd', J_1, J_2)
    H_0_tr_flat = H_0_tr.reshape((n1 * n0, n1 * n2))

    H_0_bl = np.einsum('xcdy,xaby->cdab', J_2, J_1)
    H_0_bl_flat = H_0_bl.reshape((n1 * n2, n1 * n0))

    H_0_br = np.einsum('xaby,xcdy->abcd', J_2, J_2)
    H_0_br_flat = H_0_br.reshape((n1 * n2, n1 * n2))

    H_0 = np.block([[H_0_tl_flat, H_0_tr_flat], [H_0_bl_flat, H_0_br_flat]])
    return H_0

  def compute_H1():
    H_1_tl = np.zeros((n1 * n0, n1 * n0))

    H_1_tr = np.zeros((n0 * n1, n1 * n2))
    for q, p in itertools.product(range(n0 * n1), range(n1 * n2)):
      a = int(q / n0)
      b = q % n0
      c = int(p / n1)
      d = p % n1

      if a == d:
        e_masked = np.where(prop[a] > 0, e[c], 0)
        H_1_tr[q][p] = e_masked @ x[b]

    H_1_br = np.zeros((n1 * n2, n1 * n2))

    H_1_bl = H_1_tr.transpose()

    H_1 = np.block([[H_1_tl, H_1_tr], [H_1_bl, H_1_br]])
    return H_1 / m * np.sqrt(l / l0)

  H0, H1 = compute_H0(), compute_H1()
  H = H0 + H1

  return H0, H1, H

## Plot Definitions

In [ ]:
def load_data_from_full(string, iters):
  dir_name = data_dir + string
  print("dir_name: " + dir_name)
  %cd $dir_name

  H0_eigs_arr, H1_eigs_arr, H_eigs_arr = np.empty(0), np.empty(0), np.empty(0)
  loss_array = np.empty((10, 10002))

  for i in range(0, 10):
    %cd 'id_{i}'

    losses_str = f'id_{i}_losses.json'
    with open(losses_str, 'r') as f:
      losses = np.array(json.load(f))
    loss_array[i] = losses

    x_str = f'id_{i}_x.json'
    with open(x_str, 'r') as f:
      x = np.array(json.load(f))

    y_str = f'id_{i}_y.json'
    with open(y_str, 'r') as f:
      y = np.array(json.load(f))
     
    %cd 'iters_{iters}'

    W1_str = f'id_{i}_W1.json'
    with open(W1_str, 'r') as f:
      W1 = np.array(json.load(f))
    
    W2_str = f'id_{i}_W2.json'
    with open(W2_str, 'r') as f:
      W2 = np.array(json.load(f))

    H0, H1, H = hessian(W1, W2, x, y, losses[iters], losses[0])
    eigs_H0, eigs_H1, eigs_H = npla.eigvalsh(H0), npla.eigvalsh(H1), npla.eigvalsh(H)

    H0_eigs_arr = np.concatenate((H0_eigs_arr, eigs_H0), axis=None)
    H1_eigs_arr = np.concatenate((H1_eigs_arr, eigs_H1), axis=None)
    H_eigs_arr = np.concatenate((H_eigs_arr, eigs_H), axis=None)

    %cd '../..'

  %cd $data_dir
  return H0_eigs_arr, H1_eigs_arr, H_eigs_arr, loss_array

In [ ]:
def plot_losses(losses, phi, num_iters):
  if num_iters == 0:
    mean_losses = np.average(losses[0], axis=0)
    std_losses = np.std(losses[0], axis=0)
    ci = 1.96 * std_losses / np.sqrt(3)

    set_up_plot()
    plt.scatter([0], mean_losses, label='$\phi$ = '+ str(round(phi, 1)))
    plt.legend()
    plt.show()

  else:
    mean_losses = np.average(losses[:num_iters], axis = 0)
    std_losses = np.std(losses[:num_iters], axis=0)
    ci = 1.96 * std_losses / np.sqrt(3)
  
    set_up_plot()
    plt.plot(np.arange(1, len(mean_losses) + 1), mean_losses, label='$\phi$ = '+ str(round(phi, 1)))
    plt.fill_between(np.arange(1, len(mean_losses) + 1), mean_losses + ci, mean_losses - ci, alpha=.2)
    plt.legend()
    plt.show()

In [ ]:
def plot_eigs(H0_eigs, H1_eigs, H_eigs, phi):
  plt.title("H0's Eigenvalues")
  plt.ylabel('Density')
  plt.xlabel('Eigenvalue')
  plt.hist(H0_eigs[(abs(H0_eigs) < 100) & (abs(H0_eigs) > 0.01)], bins=50, ec='black')
  #plt.savefig(string + 'phi_' + str(round(phi, 1)) + '.png')
  #files.download(string + 'phi_' + str(round(phi, 1)) + '.png')
  plt.show()
  
  plt.title("H1's Eigenvalues")
  plt.ylabel('Density')
  plt.xlabel('Eigenvalue')
  plt.hist(H1_eigs[(abs(H1_eigs) < 100) & (abs(H1_eigs) > 0.0001)], bins=50, ec='black')
  #plt.savefig(string + 'phi_' + str(round(phi, 1)) + '.png')
  #files.download(string + 'phi_' + str(round(phi, 1)) + '.png')
  plt.show()

  plt.title("H's Eigenvalues")
  plt.ylabel('Density')
  plt.xlabel('Eigenvalue')
  plt.hist(H_eigs[(abs(H_eigs) < 100) & (abs(H_eigs) > 0.0001)], bins=50, ec='black')
  #plt.savefig(string + 'phi_' + str(round(phi, 1)) + '.png')
  #files.download(string + 'phi_' + str(round(phi, 1)) + '.png')
  plt.show()

In [ ]:
def plot_negs_evol(H_eigs_dict, losses):
  perc_dict = {}
  for (num_iters, H_eigs) in H_eigs_dict.items():
    perc = len(H_eigs[H_eigs < -0.001]) / len(H_eigs)
    perc_dict[num_iters] = perc
  
  fig, ax1 = plt.subplots()

  def func(x, a, b):
      return np.log(a * np.exp(-b * x) + 1) # log(a*(e^-bx)+1)
  
  xnew = np.linspace(min(list(perc_dict.keys())), max(list(perc_dict.keys())), 200) 
  paras, pcov = curve_fit(func, list(perc_dict.keys()), list(perc_dict.values()))
  a = paras[0]
  b = paras[1]

  ynew = func(xnew, *paras)
  ax1.plot(xnew, ynew, color='blue')
  ax1.scatter(list(perc_dict.keys()), list(perc_dict.values()), color='blue')
  ax1.set_ylabel('fraction of negative eigenvalues')
  plt.xlabel('iterations of gradient descent')
  
  mean_losses = np.average(losses[:num_iters], axis = 0)
  std_losses = np.std(losses[:num_iters], axis=0)
  ci = 1.96 * std_losses / np.sqrt(3)
  ax2 = ax1.twinx()
  ax2.set_yscale('log')
  ax2.plot(np.arange(1, len(mean_losses) + 1), mean_losses, color='orange')
  ax2.fill_between(np.arange(1, len(mean_losses) + 1), mean_losses + ci, mean_losses - ci, alpha=.2, color='orange')
  ax2.set_ylabel('loss function')
  

  fig.tight_layout()
  plt.show()

# Plotting

In [ ]:
n0_widget = widgets.SelectionSlider(options=[10, 20, 50], 
                             value=20, 
                             description='n0', 
                             disabled=False, 
                             continuous_update=True, 
                             orientation='horizontal', 
                             readout=True)
n1_widget = widgets.SelectionSlider(options=[10, 20, 50], 
                             value=20, 
                             description='n1', 
                             disabled=False, 
                             continuous_update=True, 
                             orientation='horizontal', 
                             readout=True)
n2_widget = widgets.SelectionSlider(options=[10, 20, 50], 
                             value=20, 
                             description='n2', 
                             disabled=False, 
                             continuous_update=True, 
                             orientation='horizontal', 
                             readout=True)
m_widget = widgets.SelectionSlider(options=[10, 50, 200], 
                             value=50, 
                             description='m', 
                             disabled=False, 
                             continuous_update=True, 
                             orientation='horizontal', 
                             readout=True)
num_iters_widget = widgets.SelectionSlider(options=[0, 10, 100, 1000, 2500, 5000, 10000], 
                             value=2500, 
                             description='num_iters', 
                             disabled=False, 
                             continuous_update=True, 
                             orientation='horizontal', 
                             readout=True)

def plot_it(n0, n1, n2, m, num_iters):
  string = f'n0_{n0}_n1_{n1}_n2_{n2}_m_{m}'
  print('updating the plot with ' + string)
  iters_list = [0, 10, 100, 1000, 2500, 5000, 10000]
  H_eigs_dict = {} 
  H0_eigs, H1_eigs, H_eigs = None, None, None
  for iters in sorted(iters_list):
    with io.capture_output() as captured:
      H0_eigsi, H1_eigsi, H_eigsi, losses = load_data_from_full(string, iters)
    if iters == num_iters:
      H0_eigs, H1_eigs, H_eigs = H0_eigsi, H1_eigsi, H_eigsi
    H_eigs_dict[iters] = H_eigsi
  phi = ((n0 + n2) * n1) / (n2 * m)
  #plot_losses(losses, phi, num_iters)
  plot_eigs(H0_eigs, H1_eigs, H_eigs, phi)
  plot_negs_evol(H_eigs_dict, losses)

interactive(plot_it, n0=n0_widget, n1=n1_widget, n2=n2_widget, m=m_widget, num_iters=num_iters_widget)

interactive(children=(SelectionSlider(description='n0', index=1, options=(10, 20, 50), value=20), SelectionSli…